In [1]:
import sklearn
import graphviz
from sklearn.multioutput import MultiOutputRegressor
from xgboost import XGBRegressor as XGBR
import  pandas  as  pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from matplotlib.patches import Rectangle
from matplotlib.font_manager import FontProperties
import matplotlib
import pickle
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score
from sklearn.model_selection import KFold,train_test_split,GridSearchCV, cross_val_score
from sklearn import preprocessing
from sklearn.linear_model import ElasticNet, Lasso, BayesianRidge, LassoLarsIC
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.model_selection import KFold, cross_val_score,train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.feature_selection import RFECV
import xgboost as xgb
import lightgbm as lgb
from sklearn.decomposition import PCA
from xgboost import plot_tree
from time import *
import os
import cmath as ch
import scipy.io
import heapq
import hyperopt
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
import warnings
warnings.filterwarnings("ignore")
begin_time = time() # Calculate training time

OSError: [WinError 126] 找不到指定的模块。

In [ ]:
source_data = './datafile.mat'
data_info = scipy.io.loadmat(source_data)
data = data_info['obs_data']
sim_data = data_info['sim_data']
Sample_amount = sim_data.shape[0]
Scaling_factor = data_info['Scaling_factor']
release_loc = data_info['release_loc']

start_time = 11
end_time = 41
interval = end_time - start_time
percentile = 0.4
J = []
measure1 = 0
measure2 = 0
measure3 = 0
measure4 = 0
measure = []
for k in range(start_time,end_time):
    measure1 = measure1+data[k,0]
    measure2 = measure2+data[k,1]
    measure3 = measure3+data[k,2]
    measure4 = measure4+data[k,3]
measure_aver1 = measure1 / interval #测点1测量值的平均值
measure.append(measure_aver1)
measure_aver2 = measure2 / interval #测点2测量值的平均值
measure.append(measure_aver2)
measure_aver3 = measure3 / interval #测点3测量值的平均值
measure.append(measure_aver3)
measure_aver4 = measure4 / interval #测点4测量值的平均值
measure.append(measure_aver4)
measure_aver = (measure_aver1+measure_aver2+measure_aver3+measure_aver4)/4

#读取释放点的模拟数据
for j in range(Sample_amount):
    Oct4_sim = [
                    np.mean(sim_data[j,start_time:end_time,0]),
                    np.mean(sim_data[j,start_time:end_time,1]),
                    np.mean(sim_data[j,start_time:end_time,2]),
                    np.mean(sim_data[j,start_time:end_time,3])
               ]
    
    Oct4_sim_aver = np.mean(Oct4_sim)
    sum1_1=0
    sum1_2=0
    sum1_3=0
    for i in range(0,4):
        measure_bias = measure[i] - measure_aver
        measure_var = measure_bias ** 2
        Oct4_sim_bias = Oct4_sim[i] - Oct4_sim_aver
        Oct4_sim_var = Oct4_sim_bias ** 2
        temp1_1 = measure_bias * Oct4_sim_bias
        sum1_1 = sum1_1 + temp1_1
        sum1_2 = sum1_2 + Oct4_sim_var
        sum1_3 = sum1_3 + measure_var
    aver1_1 = sum1_1/4
    aver1_2 = np.sqrt(sum1_2/4)
    aver1_3 = np.sqrt(sum1_3/4) 
    J1_temp = aver1_1/(aver1_2*aver1_3)
    J.append(J1_temp)

index = J.index(max(J))

def OperCount(data,number,operator):
    num = 0
    if operator == '>':
        for i in data:
            if i > number:
                num+=1
    elif operator == '<':
        for i in data:
            if i < number:
                num+=1
    elif operator == '=':
        for i in data:
            if i == number:
                num+=1
    else:
        print('something wrong!')
    percent = num/len(data)
    return num,percent

J = list(J)
index_train = map(J.index,heapq.nlargest(int(Sample_amount*percentile),J))
index_train = list(index_train)

J = pd.DataFrame(J)
writer = pd.ExcelWriter('./Reconstruction_results/cost_function_Oct4.xls')
J.to_excel(writer,header=False,index=False)
writer.save()
writer.close()

In [ ]:
index_train = np.array(index_train)
index_train = index_train.ravel()


In [ ]:
data = pd.read_csv("./Training_datasets/train_data.csv",index_col=0) # 读取特征数据集

In [ ]:
data = data.iloc[index_train,:]
data.info()
data

In [ ]:
data_feature = data.drop(["x","y"],axis=1)

In [ ]:
data_target = data.iloc[:,24:26]
data_target_x = data.iloc[:,24:25]
data_target_y = data.iloc[:,25:26]
data_target

In [ ]:
Xtrain, Xtest, Ytrain, Ytest = train_test_split(data_feature,data_target,test_size=0.3,random_state=320)
for i in [Xtrain, Xtest, Ytrain, Ytest]:
    i.index = range(i.shape[0]) #将索引转化成0-shape[0]的整数，就可以恢复索引

Xtrain_x, Xtest_x, Ytrain_x, Ytest_x = train_test_split(data_feature,data_target_x,test_size=0.3,random_state=320)
for i in [Xtrain_x, Xtest_x, Ytrain_x, Ytest_x]:
    i.index = range(i.shape[0]) #将索引转化成0-shape[0]的整数，就可以恢复索引
 
Xtrain_y, Xtest_y, Ytrain_y, Ytest_y = train_test_split(data_feature,data_target_y,test_size=0.3,random_state=320)
for i in [Xtrain_y, Xtest_y, Ytrain_y, Ytest_y]:
    i.index = range(i.shape[0]) #将索引转化成0-shape[0]的整数，就可以恢复索引

In [ ]:
MCVA = []
MCVA_x = []
MCVA_y = []

GC = []
GC_x = []
GC_y = []
TS = []
LE = []
x_estimate = []
y_estimate = []
n_estimators = []
learning_rate = []
max_depth = []
gamma = []
subsample = []
reg_lambda = []
colsample_bytree = []
min_child_weight = []

for i in range(50):
    print('Loop time：',i)
    def huber_approx_obj(real,predict):
        d = predict -real
        h = 2 #h is delta in the formula
        scale = 1 + (d / h) ** 2
        scale_sqrt = np.sqrt(scale)
        grad = d / scale_sqrt
        hess = 1 / scale / scale_sqrt
        return grad, hess

    space={ 'n_estimators': hp.uniform ('n_estimators', 50,300),
            'learning_rate': hp.uniform('learning_rate', 0.05, 0.3),
            'max_depth': hp.quniform("max_depth", 3, 8, 1),
            'gamma': hp.uniform('gamma', 0.01,1),
            'subsample' : hp.uniform('subsample', 0.5,1),
            'reg_lambda' : hp.uniform('reg_lambda', 0.01,5),
            'colsample_bytree' : hp.uniform('colsample_bytree', 0.01,1),
            'min_child_weight' : hp.quniform('min_child_weight', 2, 10, 1),   
        }

    def hyperparameter_tuning(space):
        param = {
                 'n_estimators':     int(space['n_estimators']), 
                 'learning_rate':    space['learning_rate'], 
                 'max_depth':        int(space['max_depth']),
                 'gamma':            space['gamma'], 
                 'subsample':        space['subsample'],
                 'reg_lambda':       space['reg_lambda'],
                 'colsample_bytree': space['colsample_bytree'],
                 'min_child_weight': space['min_child_weight'],  
                 "random_state":123456
                }
        model = MultiOutputRegressor(XGBR(objective='reg:squarederror',**param)) #objective='reg:squarederror'
        r2 = cross_val_score(model,data_feature,data_target,cv=5) #scoring='neg_mean_squared_error'
        return (1-r2.mean())**2+r2.var()
    from hyperopt import tpe
    tpe_algorithm = tpe.suggest
    from hyperopt import fmin
    trials = Trials()
    best = fmin(fn=hyperparameter_tuning,
                space=space,
                algo=tpe.suggest,
                max_evals=100,
                trials=trials)


    print (best)
    end_time = time()

    run_time = end_time-begin_time

    print(run_time)
    
    learning_rate.append(best['learning_rate'])
    max_depth.append(int(best['max_depth']))
    n_estimators.append(int(best['n_estimators']))
    min_child_weight.append(best['min_child_weight'])
    subsample.append(best['subsample'])
    colsample_bytree.append(best['colsample_bytree'])
    reg_lambda.append(best['reg_lambda'])
    gamma.append(best['gamma'])
    
    param = {
                 'max_depth':        int(best['max_depth']),
                 'learning_rate':    best['learning_rate'],
                 'n_estimators':     int(best['n_estimators']),  
                 'min_child_weight': best['min_child_weight'],
                 'subsample':        best['subsample'],
                 'colsample_bytree': best['colsample_bytree'],
                 'reg_lambda':       best['reg_lambda'],
                 'gamma':            best['gamma'], 
                 "random_state":123456
                }
    x_xgbr = XGBR(objective='reg:squarederror',**param)
    scores_x = cross_val_score(x_xgbr,data_feature,data_target_x,cv=5)
    MCVA_x_temp = scores_x.mean()
    MCVA_x_temp = np.around(MCVA_x_temp, decimals = 4)
    MCVA_x.append(MCVA_x_temp)
    
    GC_x_temp = (1-scores_x.mean())**2+scores_x.var()
    GC_x_temp = np.around(GC_x_temp, decimals = 4)
    GC_x.append(GC_x_temp)
    
    x_xgbr.fit(Xtrain_x,Ytrain_x)

    y_xgbr = XGBR(objective='reg:squarederror',**param)
    scores_y = cross_val_score(y_xgbr,data_feature,data_target_y,cv=5)
    MCVA_y_temp = scores_y.mean()
    MCVA_y_temp = np.around(MCVA_y_temp, decimals = 4)
    MCVA_y.append(MCVA_y_temp)
    
    GC_y_temp = (1-scores_y.mean())**2+scores_y.var()
    GC_y_temp = np.around(GC_y_temp, decimals = 4)
    GC_y.append(GC_y_temp)
    
    y_xgbr.fit(Xtrain_y,Ytrain_y)

    multioutputregressor = MultiOutputRegressor(XGBR(objective='reg:squarederror',**param))
    scores = cross_val_score(multioutputregressor,data_feature,data_target,cv=5)
    MCVA_temp = scores.mean()
    MCVA_temp = np.around(MCVA_temp, decimals = 4)
    MCVA.append(MCVA_temp)
    
    GC_temp = (1-scores.mean())**2+scores.var()
    GC_temp = np.around(GC_temp, decimals = 4)
    GC.append(GC_temp)
    
    multioutputregressor.fit(Xtrain, Ytrain)

    print('Prediction result: x, y')
    TS_temp = multioutputregressor.score(Xtest,Ytest)
    TS_temp = np.around(TS_temp, decimals = 4)
    TS.append(TS_temp)
    
    source = pd.read_csv("./Testing_datasets/test_data.csv",index_col=0) # 12-40


    Multi_pred_source=multioutputregressor.predict(source)
    print(Multi_pred_source)
    Multi_pred_source_x=Multi_pred_source[0,0]
    Multi_pred_source_x = np.around(Multi_pred_source_x,decimals=4)
    x_estimate.append(Multi_pred_source_x)
    Multi_pred_source_y=Multi_pred_source[0,1]
    Multi_pred_source_y = np.around(Multi_pred_source_y,decimals=4)
    y_estimate.append(Multi_pred_source_y)
    LE_temp = np.sqrt((Multi_pred_source_x - 4565)**2+(Multi_pred_source_y - 7621)**2)
    LE_temp = np.around(LE_temp, decimals = 4)
    LE.append(LE_temp)
    

In [ ]:
Inversion_results = {'MCVA':MCVA,
                    'MCVA_x':MCVA_x,
                    'MCVA_y':MCVA_y,
                    'GC':GC,
                    'GC_x':GC_x,
                    'GC_y':GC_y,
                    'TS': TS,
                    'x':x_estimate,
                    'y':y_estimate,
                    'LE':LE,
                    'max_depth':max_depth,
                    'learning_rate':learning_rate,
                    'n_estimators':n_estimators,
                    'min_child_weight':min_child_weight,
                    'subsample':subsample,
                    'colsample_bytree':colsample_bytree,
                    'reg_lambda':reg_lambda,
                    'gamma':gamma
                    }
Inversion_results['x'] = np.around(Inversion_results['x'],decimals=4)
Inversion_results['y'] = np.around(Inversion_results['y'],decimals=4)
Inversion_results = pd.DataFrame(Inversion_results)
Inversion_results.to_csv('./Reconstruction_results/inversion_Oct4_before_feature_selection_50.csv')

In [ ]:
# 下面进行特征筛选
Xtrain_x, Xtest_x, Ytrain_x, Ytest_x = train_test_split(data_feature,data_target_x,test_size=0.3,random_state=320)
for i in [Xtrain_x, Xtest_x, Ytrain_x, Ytest_x]:
    i.index = range(i.shape[0]) #将索引转化成0-shape[0]的整数，就可以恢复索引
# 查看分好的训练集和测试集

Xtrain_y, Xtest_y, Ytrain_y, Ytest_y = train_test_split(data_feature,data_target_y,test_size=0.3,random_state=320)
for i in [Xtrain_y, Xtest_y, Ytrain_y, Ytest_y]:
    i.index = range(i.shape[0]) #将索引转化成0-shape[0]的整数，就可以恢复索引
#Xtrain.head()

other_params = {
                 'max_depth':        int(best['max_depth']),
                 'learning_rate':    best['learning_rate'],
                 'n_estimators':     int(best['n_estimators']),  
                 'min_child_weight': best['min_child_weight'],
                 'subsample':        best['subsample'],
                 'colsample_bytree': best['colsample_bytree'],
                 'reg_lambda':       best['reg_lambda'],
                 'gamma':            best['gamma'], 
                 "random_state":123456
}

x_xgbr = XGBR(objective='reg:squarederror',**other_params)
y_xgbr = XGBR(objective='reg:squarederror',**other_params)


selectorx = RFECV(x_xgbr,cv=5,step=1).fit(Xtrain_x,Ytrain_x)
selectory = RFECV(y_xgbr,cv=5,step=1).fit(Xtrain_y,Ytrain_y)
#check=multioutputregressor.predict(Xtest)
#multioutputregressor.score(Xtest,Ytest)#默认为Return the coefficient of determination R^2 of the prediction.

print(selectorx.support_.sum())
print(selectorx.ranking_)
idx=[]
for i in range(24):
    if selectorx.ranking_[i]!=1:
        idx.append(i)
print(idx)
X_wrapper_x=selectorx.transform(Xtrain_x)
cross_val_score(x_xgbr,X_wrapper_x,Ytrain_x,cv=5).mean()

print(selectory.support_.sum())
print(selectory.ranking_)
idy=[]
for i in range(24):
    if selectory.ranking_[i]!=1:
        idy.append(i)
print(idy)
X_wrapper_y=selectory.transform(Xtrain_y)
cross_val_score(y_xgbr,X_wrapper_y,Ytrain_y,cv=5).mean()


id_delete = list(set(idx).intersection(set(idy)))

data_feature_all = ['wave_rate1','aver1','median1','fft_shape_mean1','fft_mean1','SamEn1',
                   'wave_rate2','aver2','median2','fft_shape_mean2','fft_mean2','SamEn2',
                   'wave_rate3','aver3','median3','fft_shape_mean3','fft_mean3','SamEn3',
                   'wave_rate4','aver4','median4','fft_shape_mean4','fft_mean4','SamEn4',]
data_feature = data.drop(["x","y"],axis=1)
for i in range(len(id_delete)):
    data_feature = data_feature.drop(data_feature_all[id_delete[i]],axis=1)
print(data_feature)

In [ ]:
Xtrain, Xtest, Ytrain, Ytest = train_test_split(data_feature,data_target,test_size=0.3,random_state=320)
for i in [Xtrain, Xtest, Ytrain, Ytest]:
    i.index = range(i.shape[0]) #将索引转化成0-shape[0]的整数，就可以恢复索引

Xtrain_x, Xtest_x, Ytrain_x, Ytest_x = train_test_split(data_feature,data_target_x,test_size=0.3,random_state=320)
for i in [Xtrain_x, Xtest_x, Ytrain_x, Ytest_x]:
    i.index = range(i.shape[0]) #将索引转化成0-shape[0]的整数，就可以恢复索引
 
Xtrain_y, Xtest_y, Ytrain_y, Ytest_y = train_test_split(data_feature,data_target_y,test_size=0.3,random_state=320)
for i in [Xtrain_y, Xtest_y, Ytrain_y, Ytest_y]:
    i.index = range(i.shape[0]) #将索引转化成0-shape[0]的整数，就可以恢复索引

In [ ]:
MCVA = []
MCVA_x = []
MCVA_y = []

GC = []
GC_x = []
GC_y = []
TS = []
LE = []
x_estimate = []
y_estimate = []
x_estimate = []
y_estimate = []
n_estimators = []
learning_rate = []
max_depth = []
gamma = []
subsample = []
reg_lambda = []
colsample_bytree = []
min_child_weight = []

for i in range(50):
    print('Loop time：',i)
    def huber_approx_obj(real,predict):
        d = predict -real
        h = 2 #h is delta in the formula
        scale = 1 + (d / h) ** 2
        scale_sqrt = np.sqrt(scale)
        grad = d / scale_sqrt
        hess = 1 / scale / scale_sqrt
        return grad, hess

    space={ 'n_estimators': hp.uniform ('n_estimators', 50,300),
            'learning_rate': hp.uniform('learning_rate', 0.05, 0.3),
            'max_depth': hp.quniform("max_depth", 3, 8, 1),
            'gamma': hp.uniform('gamma', 0.01,1),
            'subsample' : hp.uniform('subsample', 0.5,1),
            'reg_lambda' : hp.uniform('reg_lambda', 0.01,5),
            'colsample_bytree' : hp.uniform('colsample_bytree', 0.01,1),
            'min_child_weight' : hp.quniform('min_child_weight', 2, 10, 1),   
        }

    def hyperparameter_tuning(space):
        param = {
                 'n_estimators':     int(space['n_estimators']), 
                 'learning_rate':    space['learning_rate'], 
                 'max_depth':        int(space['max_depth']),
                 'gamma':            space['gamma'], 
                 'subsample':        space['subsample'],
                 'reg_lambda':       space['reg_lambda'],
                 'colsample_bytree': space['colsample_bytree'],
                 'min_child_weight': space['min_child_weight'],  
                 "random_state":123456
                }
        model = MultiOutputRegressor(XGBR(objective='reg:squarederror',**param)) #objective='reg:squarederror'
        r2 = cross_val_score(model,data_feature,data_target,cv=5) #scoring='neg_mean_squared_error'
        return (1-r2.mean())**2+r2.var()

    from hyperopt import tpe
    tpe_algorithm = tpe.suggest
    from hyperopt import fmin
    trials = Trials()
    best = fmin(fn=hyperparameter_tuning,
                space=space,
                algo=tpe.suggest,
                max_evals=100,
                trials=trials)


    print (best)
    end_time = time()

    run_time = end_time-begin_time

    print(run_time)
    
    learning_rate.append(best['learning_rate'])
    max_depth.append(int(best['max_depth']))
    n_estimators.append(int(best['n_estimators']))
    min_child_weight.append(best['min_child_weight'])
    subsample.append(best['subsample'])
    colsample_bytree.append(best['colsample_bytree'])
    reg_lambda.append(best['reg_lambda'])
    gamma.append(best['gamma'])
    
    param = {
                 'max_depth':        int(best['max_depth']),
                 'learning_rate':    best['learning_rate'],
                 'n_estimators':     int(best['n_estimators']),  
                 'min_child_weight': best['min_child_weight'],
                 'subsample':        best['subsample'],
                 'colsample_bytree': best['colsample_bytree'],
                 'reg_lambda':       best['reg_lambda'],
                 'gamma':            best['gamma'], 
                 "random_state":123456
                }
    x_xgbr = XGBR(objective='reg:squarederror',**param)
    scores_x = cross_val_score(x_xgbr,data_feature,data_target_x,cv=5)
    MCVA_x_temp = scores_x.mean()
    MCVA_x_temp = np.around(MCVA_x_temp, decimals = 4)
    MCVA_x.append(MCVA_x_temp)
    
    GC_x_temp = (1-scores_x.mean())**2+scores_x.var()
    GC_x_temp = np.around(GC_x_temp, decimals = 4)
    GC_x.append(GC_x_temp)
    
    x_xgbr.fit(Xtrain_x,Ytrain_x)

    y_xgbr = XGBR(objective='reg:squarederror',**param)
    scores_y = cross_val_score(y_xgbr,data_feature,data_target_y,cv=5)
    MCVA_y_temp = scores_y.mean()
    MCVA_y_temp = np.around(MCVA_y_temp, decimals = 4)
    MCVA_y.append(MCVA_y_temp)
    
    GC_y_temp = (1-scores_y.mean())**2+scores_y.var()
    GC_y_temp = np.around(GC_y_temp, decimals = 4)
    GC_y.append(GC_y_temp)
    
    y_xgbr.fit(Xtrain_y,Ytrain_y)

    multioutputregressor = MultiOutputRegressor(XGBR(objective='reg:squarederror',**param))
    scores = cross_val_score(multioutputregressor,data_feature,data_target,cv=5)
    MCVA_temp = scores.mean()
    MCVA_temp = np.around(MCVA_temp, decimals = 4)
    MCVA.append(MCVA_temp)
    
    GC_temp = (1-scores.mean())**2+scores.var()
    GC_temp = np.around(GC_temp, decimals = 4)
    GC.append(GC_temp)
    
    multioutputregressor.fit(Xtrain, Ytrain)

    TS_temp = multioutputregressor.score(Xtest,Ytest)
    TS_temp = np.around(TS_temp, decimals = 4)
    TS.append(TS_temp)
    # 对测点的数据进行验证
    source_feature_all = ['wave_rate1','aver1','median1','fft_shape_mean1','fft_mean1','SamEn1',
                   'wave_rate2','aver2','median2','fft_shape_mean2','fft_mean2','SamEn2',
                   'wave_rate3','aver3','median3','fft_shape_mean3','fft_mean3','SamEn3',
                   'wave_rate4','aver4','median4','fft_shape_mean4','fft_mean4','SamEn4',]
    source = pd.read_csv("./Testing_datasets/test_data.csv",index_col=0) # 12-40
    for i in range(len(id_delete)):
        source = source.drop(source_feature_all[id_delete[i]],axis=1)

    Multi_pred_source=multioutputregressor.predict(source)
    print(Multi_pred_source)
    Multi_pred_source_x=Multi_pred_source[0,0]
    Multi_pred_source_x = np.around(Multi_pred_source_x,decimals=4)
    x_estimate.append(Multi_pred_source_x)
    Multi_pred_source_y=Multi_pred_source[0,1]
    Multi_pred_source_y = np.around(Multi_pred_source_y,decimals=4)
    y_estimate.append(Multi_pred_source_y)
    LE_temp = np.sqrt((Multi_pred_source_x - 4565)**2+(Multi_pred_source_y - 7621)**2)
    LE_temp = np.around(LE_temp, decimals = 4)
    LE.append(LE_temp)

In [ ]:
Inversion_results = {'MCVA':MCVA,
                    'MCVA_x':MCVA_x,
                    'MCVA_y':MCVA_y,
                    'GC':GC,
                    'GC_x':GC_x,
                    'GC_y':GC_y,
                    'TS': TS,
                    'x':x_estimate,
                    'y':y_estimate,
                    'LE':LE,
                    'max_depth':max_depth,
                    'learning_rate':learning_rate,
                    'n_estimators':n_estimators,
                    'min_child_weight':min_child_weight,
                    'subsample':subsample,
                    'colsample_bytree':colsample_bytree,
                    'reg_lambda':reg_lambda,
                    'gamma':gamma
                    }
Inversion_results['x'] = np.around(Inversion_results['x'],decimals=4)
Inversion_results['y'] = np.around(Inversion_results['y'],decimals=4)
Inversion_results = pd.DataFrame(Inversion_results)
Inversion_results.to_csv('./Reconstruction_results/inversion_Oct4_after_feature_selection_50.csv')

In [ ]:
# 置信区间统计
from scipy import stats
inversion_before_FS  = pd.read_csv('./Reconstruction_results/inversion_Oct4_before_feature_selection_50.csv',index_col=0)
inversion_before_FS_x_aver = np.mean(inversion_before_FS['x'].values)
inversion_before_FS_x_std = np.std(inversion_before_FS['x'].values)
inversion_before_FS_x_low = inversion_before_FS_x_aver - 1.96 * inversion_before_FS_x_std
inversion_before_FS_x_high = inversion_before_FS_x_aver + 1.96 * inversion_before_FS_x_std

inversion_before_FS_y_aver = np.mean(inversion_before_FS['y'].values)
inversion_before_FS_y_std = np.std(inversion_before_FS['y'].values)
inversion_before_FS_y_low = inversion_before_FS_y_aver - 1.96 * inversion_before_FS_y_std
inversion_before_FS_y_high = inversion_before_FS_y_aver + 1.96 * inversion_before_FS_y_std

print("Before feature selection")
print("x interval: [", inversion_before_FS_x_low, "," ,inversion_before_FS_x_high, "]")
print("y interval: [", inversion_before_FS_y_low, "," ,inversion_before_FS_y_high, "]")



In [ ]:
inversion_after_FS  = pd.read_csv('./Reconstruction_results/inversion_Oct4_after_feature_selection_50.csv',index_col=0)
inversion_after_FS_x_aver = np.mean(inversion_after_FS['x'].values)
inversion_after_FS_x_std = np.std(inversion_after_FS['x'].values)
inversion_after_FS_x_low = inversion_after_FS_x_aver - 1.96 * inversion_after_FS_x_std
inversion_after_FS_x_high = inversion_after_FS_x_aver + 1.96 * inversion_after_FS_x_std

inversion_after_FS_y_aver = np.mean(inversion_after_FS['y'].values)
inversion_after_FS_y_std = np.std(inversion_after_FS['y'].values)
inversion_after_FS_y_low = inversion_after_FS_y_aver - 1.96 * inversion_after_FS_y_std
inversion_after_FS_y_high = inversion_after_FS_y_aver + 1.96 * inversion_after_FS_y_std

print("After feature selection")
print("x interval: [", inversion_after_FS_x_low, "," ,inversion_after_FS_x_high, "]")
print("y interval: [", inversion_after_FS_y_low, "," ,inversion_after_FS_y_high, "]")



In [ ]:
#### 计算特征重要性
source_feature_all = ['wave_rate1','aver1','median1','fft_shape_mean1','fft_mean1','SamEn1',
                   'wave_rate2','aver2','median2','fft_shape_mean2','fft_mean2','SamEn2',
                   'wave_rate3','aver3','median3','fft_shape_mean3','fft_mean3','SamEn3',
                   'wave_rate4','aver4','median4','fft_shape_mean4','fft_mean4','SamEn4',]
source_feature_all
new_list = [source_feature_all[i] for i in range(len(source_feature_all)) if i not in id_delete]


In [ ]:
font_set = FontProperties(fname=r"C:\Windows\Fonts\simhei.ttf", size=60)
font_set2 = FontProperties(fname=r"C:\Windows\Fonts\simhei.ttf", size=60)
feature_importances0=multioutputregressor.estimators_[0].feature_importances_
feature_importances1=multioutputregressor.estimators_[1].feature_importances_

fig, ax = plt.subplots(figsize=(15, 10))
ind = np.arange(len(new_list))
width = 0.3
rect1 = ax.barh(ind - width/2, feature_importances0,width,label='x',color='y')
rect1 = ax.barh(ind + width/2, feature_importances1,width,label='y',color='purple')
plt.tick_params(labelsize=30)
ax.set_yticks(ind + width / 2)
ax.set_yticklabels(new_list)
ax.set_xlabel('Feature importance',fontsize = 35)
ax.legend(fontsize=30)

plt.show()

fig.savefig("./Reconstruction_results/Feature_importance_Oct4.png",dpi=300)